# Classification Artificial Neural Network (Defense)

### Importing libraries and reading data

In [ ]:
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pytz
import datetime
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import tensorflow as tf
from sklearn.model_selection import KFold

In [2]:
df_train = pd.read_csv('df_train_defense.csv')
df_test = pd.read_csv('df_test_defense.csv')

In [3]:
X_train = df_train.drop('dist_RM_BA', axis=1)
y_train = df_train['dist_RM_BA']
X_test = df_test.drop(['dist_RM_BA'], axis=1)
y_test = df_test['dist_RM_BA']

In [4]:
round(1-y_train.value_counts()[1] / y_train.value_counts()[0],3)

0.848

### Scaling

In [4]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.values
y_test = y_test.values

### Training the Model

In [6]:
metrics = pd.DataFrame(columns=['fold', 'accuracy', 'precision', 'recall', 'f1_score'])

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

#with tf.device('/cpu:0'):
# K-fold Cross Validation model evaluation
fold_no = 1
for train, val in kfold.split(X_train, y_train):

    # Define the model architecture
    model = Sequential()

    model.add(Dense(units=36,activation='relu'))
    model.add(Dropout(0.2))

    #hidden layers
    model.add(Dense(units=30,activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=24,activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=18,activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=12,activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=6,activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=1,activation='sigmoid'))

    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

    # Compile the model
    # For a binary classification problem
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(X_train[train], y_train[train],
          epochs=1000,
          batch_size=1,
          validation_data=(X_train[val], y_train[val]), 
          callbacks=[early_stop])

    # Generate generalization metrics
    y_pred = (model.predict(X_test) > 0.5).astype("int32")

    scores = [fold_no,accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), 
              recall_score(y_test, y_pred), f1_score(y_test, y_pred)]

    metrics.loc[len(metrics)] = scores

    print(classification_report(y_test,y_pred))
    # Increase fold number
    fold_no = fold_no + 1


metrics.loc[len(metrics)] = ['mean', metrics['accuracy'].mean(), metrics['precision'].mean(), 
                             metrics['recall'].mean(), metrics['f1_score'].mean()]
metrics.loc[len(metrics)] = ['std', metrics['accuracy'].iloc[:-1].std(), metrics['precision'].iloc[:-1].std(), 
                             metrics['recall'].iloc[:-1].std(), metrics['f1_score'].iloc[:-1].std()]  
metrics = metrics.set_index('fold')

tz = pytz.timezone('Brazil/East')
metrics.to_csv( f'defense_{datetime.datetime.now(tz).strftime("%d-%m-%Y-%H:%M")}.csv')


2021-09-04 01:57:00.136394: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-04 01:57:00.194410: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-04 01:57:00.211841: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/1000
6800/6800 [==============================] - 13s 2ms/step - loss: 0.3747 - accuracy: 0.8566 - precision: 0.1069 - recall: 0.0096 - val_loss: 0.2304 - val_accuracy: 0.8635 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/1000
6800/6800 [==============================] - 12s 2ms/step - loss: 0.2225 - accuracy: 0.8760 - precision: 0.1943 - recall: 0.0241 - val_loss: 0.2144 - val_accuracy: 0.9118 - val_precision: 0.6737 - val_recall: 0.6853
Epoch 3/1000
6800/6800 [==============================] - 13s 2ms/step - loss: 0.2016 - accuracy: 0.9105 - precision: 0.6331 - recall: 0.6501 - val_loss: 0.2098 - val_accuracy: 0.9171 - val_precision: 0.6784 - val_recall: 0.7457
Epoch 4/1000
6800/6800 [==============================] - 13s 2ms/step - loss: 0.1951 - accuracy: 0.9193 - precision: 0.6725 - recall: 0.7813 - val_loss: 0.2031 - val_accuracy: 0.9253 - val_precision: 0.7215 

In [7]:
metrics

,accuracy,precision,recall,f1_score
fold,,,,
1.0,0.922667,0.679426,0.743455,0.710000
2.0,0.912000,0.626609,0.764398,0.688679
3.0,0.924667,0.721591,0.664921,0.692098
4.0,0.906000,0.611607,0.717277,0.660241
5.0,0.928667,0.808824,0.575916,0.672783
mean,0.918800,0.689611,0.693194,0.684760
std,0.009445,0.079674,0.075382,0.019045
